In [3]:
#input the info needed to create a connection between sql server and this notebook using sqlalchemy
import getpass
mypasswd = getpass.getpass()
username = 'sebmb'
host = 'pgsql.dsa.lan'
database = 'dsa_student'

········


In [4]:
#create a connection and delete my password once connected
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

postgres_db = {'drivername': 'postgres',
               'username': username,
               'password': mypasswd,
               'host': host,
               'database' :database}

engine = create_engine(URL(**postgres_db), echo=True)

del mypasswd

In [5]:
#load in the file and clean up the data into a pandas dataframe
import pandas as pd

file = 'superbowl.csv'

sb_df = pd.read_csv(file,  encoding='latin-1')
sb_df.columns = [x.lower() for x in sb_df.columns]
sb_df = sb_df.rename(columns={'winner pts' : 'winner_pts', 'loser pts' : 'loser_pts'})
sb_df = sb_df.dropna()
sb_df.head()

,date,sb,winner,winner_pts,loser,loser_pts,mvp,stadium,city,state
0,Feb 2 2020,LIV (54),Kansas City Chiefs,31,San Francisco 49ers,20,Patrick Mahomes,Hard Rock Stadium,Miami Gardens,Florida
1,Feb 3 2019,LIII (53),New England Patriots,13,Los Angeles Rams,3,Julian Edelman,Mercedes-Benz Stadium,Atlanta,Georgia
2,Feb 4 2018,LII (52),Philadelphia Eagles,41,New England Patriots,33,Nick Foles,U.S. Bank Stadium,Minneapolis,Minnesota
3,Feb 5 2017,LI (51),New England Patriots,34,Atlanta Falcons,28,Tom Brady,NRG Stadium,Houston,Texas
4,Feb 7 2016,50,Denver Broncos,24,Carolina Panthers,10,Von Miller,Levi's Stadium,Santa Clara,California


In [6]:
#move the csv file from pandas to sql to query the results in the next few cells
sb_df.to_sql('sb_df', # The table to load
          engine,             # The engine created above
          schema= username,   # The schema where the table lives, our pawprint
          if_exists='replace', # If the table is found, it would keep loading the end of table.
          index=False,        # Recall that panda data frame has a row index, so we need to ignore it
          chunksize=50) 

2021-12-27 19:12:25,328 INFO sqlalchemy.engine.base.Engine select version()
2021-12-27 19:12:25,329 INFO sqlalchemy.engine.base.Engine {}
2021-12-27 19:12:25,331 INFO sqlalchemy.engine.base.Engine select current_schema()
2021-12-27 19:12:25,332 INFO sqlalchemy.engine.base.Engine {}
2021-12-27 19:12:25,334 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-12-27 19:12:25,335 INFO sqlalchemy.engine.base.Engine {}
2021-12-27 19:12:25,337 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-12-27 19:12:25,337 INFO sqlalchemy.engine.base.Engine {}
2021-12-27 19:12:25,339 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2021-12-27 19:12:25,339 INFO sqlalchemy.engine.base.Engine {}
2021-12-27 19:12:25,342 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2021-12-27 19:12:25

Below this cell is code copy and pasted from my personal server through Mizzou's SQL server. I give samples of different queries that can be done using the given datatable.

This is a preview of what the table looks like in SQL

SELECT * FROM sebmb.sb_df LIMIT 2;

    date    |    sb     |        winner        | winner_pts |        loser        | loser_pts |       mvp       |        stad
ium        |     city      |  state
------------+-----------+----------------------+------------+---------------------+-----------+-----------------+------------
-----------+---------------+---------
 Feb 2 2020 | LIV (54)  | Kansas City Chiefs   |         31 | San Francisco 49ers |        20 | Patrick Mahomes | Hard Rock S
tadium     | Miami Gardens | Florida
 Feb 3 2019 | LIII (53) | New England Patriots |         13 | Los Angeles Rams    |         3 | Julian Edelman  | Mercedes-Be
nz Stadium | Atlanta       | Georgia
(2 rows)

These are the top 5 highest scoring superbowls, the team that won and the points they scored.

SELECT sb, winner, MAX(winner_pts) FROM sebmb.sb_df GROUP BY sb,winner ORDER BY MAX(winner_pts) DESC LIMIT 5;
     sb      |        winner        | max
-------------+----------------------+-----
 XXIV (24)   | San Francisco 49ers  |  55
 XXVII (27)  | Dallas Cowboys       |  52
 XXIX (29)   | San Francisco 49ers  |  49
 XXXVII (37) | Tampa Bay Buccaneers |  48
 XX (20)     | Chicago Bears        |  46
(5 rows)

This query shows the teams that have won a superbowl and how many times they have won.

SELECT winner, COUNT(*) FROM sebmb.sb_df GROUP BY winner ORDER BY COUNT(*) DESC;

        winner        | count
----------------------+-------
 New England Patriots |     6
 Pittsburgh Steelers  |     6
 Dallas Cowboys       |     5
 San Francisco 49ers  |     5
 New York Giants      |     4
 Green Bay Packers    |     4
 Denver Broncos       |     3
 Washington Redskins  |     3
 Baltimore Ravens     |     2
 Kansas City Chiefs   |     2
 Oakland Raiders      |     2
 Miami Dolphins       |     2
 Philadelphia Eagles  |     1
 New Orleans Saints   |     1
 Indianapolis Colts   |     1
 Chicago Bears        |     1
 Tampa Bay Buccaneers |     1
 Seattle Seahawks     |     1
 St. Louis Rams       |     1
 Los Angeles Raiders  |     1
 New York Jets        |     1
 Baltimore Colts      |     1
(22 rows)

These are the states that have hosted a superbowl and the average points put up by the winning team.

SELECT state, ROUND(AVG(winner_pts)) FROM sebmb.sb_df GROUP BY state ORDER BY AVG(winner_pts) DESC;

   state    | round
------------+-------
 New Jersey |    43
 Minnesota  |    39
 California |    34
 Louisiana  |    31
 Texas      |    30
 Florida    |    29
 Arizona    |    24
 Michigan   |    24
 Georgia    |    22
 Indiana    |    21
(10 rows)

This nested query displays the name of the mvp when the winning team put up at least 30 pts.

SELECT mvp FROM sebmb.sb_df WHERE winner_pts IN (SELECT winner_pts FROM sebmb.sb_df WHERE winner_pts > 30);

        mvp
-------------------
 Patrick Mahomes
 Nick Foles
 Tom Brady
 Malcolm Smith
 Joe Flacco
 Aaron Rodgers
 Drew Brees
 Tom Brady
 Dexter Jackson
 Ray Lewis+
 John Elway+
 Terrell Davis+
 Desmond Howard
 Steve Young+
 Troy Aikman+
 Mark Rypien
 Joe Montana+
 Doug Williams
 Phil Simms
 Richard Dent+
 Joe Montana+
 Marcus Allen+
 Terry Bradshaw+
 Terry Bradshaw+
 Fred Biletnikoff+
 Bart Starr+
 Bart Starr+
(27 rows)